In [1]:
import numpy as np
import pandas as pd
from ipywidgets import widgets
from sklearn import preprocessing
import sklearn
from datetime import time
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
from scipy.ndimage.filters import gaussian_filter
import plotly.graph_objects as go
from collections import Counter
import math

In [2]:
nDrives = 50

In [2]:
drivePaths = [str(path) for path in Path(".").rglob("outputs/withNewFeatures/dataByLocation*.csv")]
# drivePaths = ["./outputs/withNewFeatures/dataByLocation_2021-06-08-15-01-31_2T3Y1RFV8KC014025.csv"]

In [4]:
subsamplingPeriod = 1

drivesWithLocation = []
drivesWithoutLocation = []
driveIDs = []
for drivePath in drivePaths:
    drive = pd.read_csv(drivePath)
    if len(drive) > 1200 and "dataByLocation_2021-06-03-19-38-35_2T3Y1RFV8KC014025.csv" not in drivePath:
        driveIDs.append("_".join(drivePath.split("/")[-1].split("_")[1:-1]))
        drive = drive.iloc[::subsamplingPeriod]
        driveWithoutLocation = drive.drop(columns=["Time","Longitude", "Latitude"])
#         driveWithoutLocation = driveWithoutLocation.drop(columns=["ZAcceleration", "LongAcceleration", "LatAcceleration"])
        drivesWithLocation.append(drive)
        drivesWithoutLocation.append(driveWithoutLocation)
print("Found", len(drivesWithoutLocation), "drives that meet specifications")

Found 1 drives that meet specifications


## Normalize Data

In [8]:
scalerFile = open('otherLargeFiles/scalerUsedForTrainingInputs.pkl', 'rb')
scaler = pickle.load(scalerFile)                     
scalerFile.close()

normalizedDrives = []
for drive in drivesWithoutLocation:
    drive = drive.values[:]
    data_normalized = scaler.transform(drive)
    data_normalized = pd.DataFrame(data_normalized)
    data_normalized[0] = drive[:,0]
    normalizedDrives.append(data_normalized)

## Window (discrete) and Generate Labels

In [9]:
# Faster
sequenceLength = 10

windowsPerDrive = []
for drive in normalizedDrives:
    nWindows = math.floor(len(drive) / sequenceLength)
    if len(drive) % sequenceLength == 0:
        nWindows-=1
    windowsPerDrive.append(nWindows)
    
datasetLength = sum(windowsPerDrive)
dataset = {"samples":np.full((datasetLength, sequenceLength, len(normalizedDrives[0].columns)), -1.),"labels":np.full((datasetLength, 2), -1.)}
datasetIndex = 0

for k,drive in enumerate(normalizedDrives):
    nWindows = windowsPerDrive[k]
    print("on drive",k, " Has", nWindows, "windows worth of samples")
    for i in range(nWindows):
        windowStartIndex = i * sequenceLength
        windowEndIndex = windowStartIndex + sequenceLength
        window = drive.iloc[windowStartIndex:windowEndIndex].to_numpy()
        startWithLocation = drivesWithLocation[k].iloc[windowStartIndex]
        startLong = startWithLocation.Longitude
        startLat = startWithLocation.Latitude
        nextWithLocation = drivesWithLocation[k].iloc[windowEndIndex]
        nextLong = nextWithLocation.Longitude
        nextLat = nextWithLocation.Latitude
        deltaLong = nextLong - startLong
        deltaLat = nextLat - startLat
        deltas = [deltaLong, deltaLat]
        dataset["samples"][datasetIndex] = window
        dataset["labels"][datasetIndex] = deltas
        datasetIndex+=1

on drive 0  Has 1800 windows worth of samples


### Normalize labels

In [11]:
scalerFile = open('otherLargeFiles/scalerUsedForTrainingLabels.pkl', 'rb')
scaler = pickle.load(scalerFile)                     
scalerFile.close()

originalLabels = dataset["labels"]
labels_normalized = scaler.transform(originalLabels)
dataset["labels"] = labels_normalized

### Pickle

In [13]:
nextTestNumber = max([int(str(path).split(".")[0].split("-")[-1]) for path in Path("./otherLargeFiles").rglob("test-drive*")], default=-1) + 1
dbfile = open(f'otherLargeFiles/test-drive-{str(nextTestNumber)}.pkl', 'ab')

# source, destination
pickle.dump(dataset, dbfile)                     
dbfile.close()

MORE

In [41]:
def createTestDrive(path):
    drivePaths = [path]
    subsamplingPeriod = 1

    drivesWithLocation = []
    drivesWithoutLocation = []
    driveIDs = []
    for drivePath in drivePaths:
        drive = pd.read_csv(drivePath)
        if len(drive) > 1200 and "dataByLocation_2021-06-03-19-38-35_2T3Y1RFV8KC014025.csv" not in drivePath:
            driveIDs.append("_".join(drivePath.split("/")[-1].split("_")[1:-1]))
            drive = drive.iloc[::subsamplingPeriod]
            driveWithoutLocation = drive.drop(columns=["Time","Longitude", "Latitude"])
    #         driveWithoutLocation = driveWithoutLocation.drop(columns=["ZAcceleration", "LongAcceleration", "LatAcceleration"])
            drivesWithLocation.append(drive)
            drivesWithoutLocation.append(driveWithoutLocation)
    print("Found", len(drivesWithoutLocation), "drives that meet specifications")
    if len(drivesWithoutLocation) > 0:
        scalerFile = open('otherLargeFiles/scalerUsedForTrainingInputs.pkl', 'rb')
        scaler = pickle.load(scalerFile)                     
        scalerFile.close()

        normalizedDrives = []
        for drive in drivesWithoutLocation:
            drive = drive.values[:]
            data_normalized = scaler.transform(drive)
            data_normalized = pd.DataFrame(data_normalized)
            data_normalized[0] = drive[:,0]
            normalizedDrives.append(data_normalized)

        # Faster
        sequenceLength = 10

        windowsPerDrive = []
        for drive in normalizedDrives:
            nWindows = math.floor(len(drive) / sequenceLength)
            if len(drive) % sequenceLength == 0:
                nWindows-=1
            windowsPerDrive.append(nWindows)

        datasetLength = sum(windowsPerDrive)
        dataset = {"samples":np.full((datasetLength, sequenceLength, len(normalizedDrives[0].columns)), -1.),"labels":np.full((datasetLength, 2), -1.)}
        datasetIndex = 0

        for k,drive in enumerate(normalizedDrives):
            nWindows = windowsPerDrive[k]
            print("on drive",k, " Has", nWindows, "windows worth of samples")
            for i in range(nWindows):
                windowStartIndex = i * sequenceLength
                windowEndIndex = windowStartIndex + sequenceLength
                window = drive.iloc[windowStartIndex:windowEndIndex].to_numpy()
                startWithLocation = drivesWithLocation[k].iloc[windowStartIndex]
                startLong = startWithLocation.Longitude
                startLat = startWithLocation.Latitude
                nextWithLocation = drivesWithLocation[k].iloc[windowEndIndex]
                nextLong = nextWithLocation.Longitude
                nextLat = nextWithLocation.Latitude
                deltaLong = nextLong - startLong
                deltaLat = nextLat - startLat
                deltas = [deltaLong, deltaLat]
                dataset["samples"][datasetIndex] = window
                dataset["labels"][datasetIndex] = deltas
                datasetIndex+=1
        scalerFile = open('otherLargeFiles/scalerUsedForTrainingLabels.pkl', 'rb')
        scaler = pickle.load(scalerFile)                     
        scalerFile.close()

        originalLabels = dataset["labels"]
        labels_normalized = scaler.transform(originalLabels)
        dataset["labels"] = labels_normalized
        nextTestNumber = max([int(str(path).split(".")[0].split("-")[-1]) for path in Path("./otherLargeFiles/AllDrives").rglob("drive*")], default=-1) + 1
        dbfile = open(f'otherLargeFiles/AllDrives/drive-{driveIDs[0]}.pkl', 'ab')

        # source, destination
        pickle.dump(dataset, dbfile)                     
        dbfile.close()

In [42]:
drivesNotInDataset = []
drivesInDatasetFile = open(f'otherLargeFiles/drives-used.pkl', 'rb')
drivesInDataset = pickle.load(drivesInDatasetFile)
drivesInDatasetFile.close()
for drivePath in drivePaths:
    if drivePath not in drivesInDataset:
        drivesNotInDataset.append(drivePath)

In [43]:

for path in [str(path) for path in Path(".").rglob("outputs/withNewFeatures/dataByLocation*.csv")]:
    createTestDrive(path)

Found 1 drives that meet specifications
on drive 0  Has 1779 windows worth of samples


/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from ve

Found 0 drives that meet specifications
Found 0 drives that meet specifications
Found 1 drives that meet specifications
on drive 0  Has 1005 windows worth of samples
Found 1 drives that meet specifications
on drive 0  Has 2929 windows worth of samples


/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from ve

Found 0 drives that meet specifications
Found 0 drives that meet specifications
Found 0 drives that meet specifications
Found 1 drives that meet specifications
on drive 0  Has 700 windows worth of samples
Found 1 drives that meet specifications
on drive 0  Has 1736 windows worth of samples


...

/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from ve

Found 1 drives that meet specifications
on drive 0  Has 647 windows worth of samples
Found 1 drives that meet specifications
on drive 0  Has 1093 windows worth of samples


/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Found 1 drives that meet specifications
on drive 0  Has 1731 windows worth of samples
Found 1 drives that meet specifications
on drive 0  Has 666 windows worth of samples


/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/isaac/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
